In [15]:
import pandas as pd
import numpy as np
import pip
import os
from tqdm.notebook import tqdm, trange
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
try:
    __import__("xarray")
except ImportError:
    pip.main(['install', "xarray"])

import xarray as xr
pd.options.display.max_columns = None
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.transform import factor_cmap, factor_mark


pd.options.display.max_columns = None
pd.options.display.max_rows = None


import geopandas
import folium
from folium import plugins
import geodatasets


#import warnings
#warnings.filterwarnings('ignore')


In [16]:
weather = pd.read_csv('../data/weather/weather_cleaned.csv')

In [17]:
weather.head()

,Date,Station,Name,Latitude,Longitude,Elevation,AverageDailyWind,NumDaysPrecipAvg,FastestWindTime,MultidayPrecipTotal,PeakGustTime,Precipitation,Snowfall,MinSoilTemp,TimeAvgTemp,TimeMaxTemp,TimeMinTemp,TempAtObs,2MinMaxWindDirection,5MinMaxWindDirection,2MinMaxWindSpeed,5MinMaxWindSpeed,Fog,Heavy Fog,Thunder,Ice Pellets,Hail,Rime,Smoke,Tornado,High Wind,Mist,Drizzle,Rain,Snow,Ground Fog
0,2023-09-29,USW00012930,"NEW ORLEANS AUDUBON, LA US",29.91660,-90.130200,6.1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,85.0,73.0,73.0,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2023-09-29,USW00012916,"NEW ORLEANS AIRPORT, LA US",29.99755,-90.277720,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2023-09-29,US1LAOR0014,"NEW ORLEANS 3.8 WSW, LA US",29.93772,-90.131310,2.1,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2023-09-29,US1LAOR0009,"NEW ORLEANS 5.0 N, LA US",30.01515,-90.065586,0.6,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2023-09-28,USW00053917,"NEW ORLEANS LAKEFRONT AIRPORT, LA US",30.04934,-90.028990,0.9,15.43,NaN,NaN,NaN,1815.0,0.0,NaN,NaN,NaN,87.0,77.0,NaN,60.0,90.0,21.9,25.9,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [18]:
stations = weather[["Station", "Name", "Latitude", "Longitude"]].drop_duplicates().set_index("Station")

In [19]:
stations

,Name,Latitude,Longitude
Station,,,
USW00012930,"NEW ORLEANS AUDUBON, LA US",29.916600,-90.130200
USW00012916,"NEW ORLEANS AIRPORT, LA US",29.997550,-90.277720
US1LAOR0014,"NEW ORLEANS 3.8 WSW, LA US",29.937720,-90.131310
US1LAOR0009,"NEW ORLEANS 5.0 N, LA US",30.015150,-90.065586
USW00053917,"NEW ORLEANS LAKEFRONT AIRPORT, LA US",30.049340,-90.028990
US1LAOR0006,"NEW ORLEANS 2.1 ENE, LA US",29.961679,-90.038803
US1LAOR0019,"NEW ORLEANS 2.8 E, LA US",29.944803,-90.025386
US1LAOR0003,"NEW ORLEANS 3.6 SW, LA US",29.919548,-90.118474
US1LAOR0012,"NEW ORLEANS 4.8 N, LA US",30.019430,-90.068660


In [20]:
station_map = folium.Map(location=[30, -90.15], tiles="OpenStreetMap", zoom_start=10.5)

for idx, station in stations.iterrows():
    name = str(station["Name"])
    station_lat = float(station["Latitude"])
    station_lon = float(station["Longitude"])

    
    folium.Marker([station_lat, station_lon], tooltip = name).add_to(station_map)

station_map

In [21]:
# location of data 
data_folder = '../data/calls_for_service'
# paths for all csv files in data_folder
csv_files = [f for f in os.listdir(data_folder) if f.endswith('csv')]
# if compiled csv file does not already exist
if 'calls_master.csv' not in csv_files:
    # make empty dataframe
    calls_for_service = pd.DataFrame()
    # combine all files in folder into one large dataframe
    for f in tqdm(csv_files, desc = "Combining Files"):
        file_path = os.path.join(data_folder, f)
        df = pd.read_csv(file_path)
        calls_for_service = pd.concat([calls_for_service, df], ignore_index = True)
    # export to combined csv file
    calls_for_service.to_csv('../data/calls_for_service/calls_master.csv')
else:
    # if compiled csv already exists, just load that
    calls_for_service = pd.read_csv(os.path.join(data_folder, 'calls_master.csv'))


/var/folders/vh/j0_lsnhd1_d199g2tx8chgx00000gp/T/ipykernel_72244/563881976.py:18: DtypeWarning: Columns (2,5,6,7,12,16,17,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  calls_for_service = pd.read_csv(os.path.join(data_folder, 'calls_master.csv'))


In [22]:
calls_for_service.head()

,Unnamed: 0,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,Type_,TimeArrival
0,0,A3472220,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3688756.0,528696.0,01/28/2020 01:37:20 AM,01/28/2020 01:37:20 AM,01/28/2020 01:37:28 AM,01/28/2020 02:25:50 AM,NAT,Necessary Action Taken,N,4G04,Atlantic Ave & Slidell St,70114.0,4,POINT (-90.04525645 29.94750953),NaN,NaN
1,1,A0000220,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3668710.0,533007.0,01/01/2020 12:00:42 AM,01/01/2020 12:00:42 AM,01/01/2020 12:00:42 AM,01/01/2020 01:37:16 AM,NAT,Necessary Action Taken,Y,2U04,034XX Broadway St,70125.0,2,POINT (-90.10840522 29.95996774),NaN,NaN
2,2,A2190820,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3682445.0,530709.0,01/17/2020 09:18:41 PM,01/17/2020 09:18:41 PM,01/17/2020 09:18:47 PM,01/17/2020 09:18:54 PM,NAT,Necessary Action Taken,N,8B02,N Peters St & Bienville St,70130.0,8,POINT (-90.065113 29.95323762),NaN,NaN
3,3,A2874820,21,COMPLAINT OTHER,2A,21,COMPLAINT OTHER,1J,3737616.0,590067.0,01/23/2020 10:19:48 AM,01/23/2020 10:22:05 AM,01/23/2020 10:31:11 AM,01/23/2020 10:34:35 AM,GOA,GONE ON ARRIVAL,N,7L08,I-10 E,70129.0,7,POINT (-89.88854843 30.11465463),NaN,NaN
4,4,A2029120,34S,AGGRAVATED BATTERY BY SHOOTING,2C,34S,AGGRAVATED BATTERY BY SHOOTING,2C,3696210.0,551411.0,01/16/2020 05:09:05 PM,01/16/2020 05:09:43 PM,01/16/2020 05:16:07 PM,01/16/2020 10:49:37 PM,RTF,REPORT TO FOLLOW,N,7A01,Chef Menteur Hwy & Downman Rd,70126.0,7,POINT (-90.02090137 30.00973449),NaN,NaN


In [30]:
tqdm.pandas()

def extract_location(row):
    ycoord, xcoord = calls_for_service["Location"].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')# convert to numeric
    return ycoord, xcoord

# regex to extract latitude and longitude from location column
calls_for_service[["ycoord", "xcoord"]] = calls_for_service.apply(extract_location, result_type ='expand', axis = 1)

calls_for_service



KeyboardInterrupt: 

In [31]:
calls_for_service[["ycoord", "xcoord"]] = calls_for_service["Location"].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')

In [32]:
calls_for_service.head()

,Unnamed: 0,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,Type_,TimeArrival,ycoord,xcoord
0,0,A3472220,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3688756.0,528696.0,01/28/2020 01:37:20 AM,01/28/2020 01:37:20 AM,01/28/2020 01:37:28 AM,01/28/2020 02:25:50 AM,NAT,Necessary Action Taken,N,4G04,Atlantic Ave & Slidell St,70114.0,4,POINT (-90.04525645 29.94750953),NaN,NaN,-90.04525645,29.94750953
1,1,A0000220,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3668710.0,533007.0,01/01/2020 12:00:42 AM,01/01/2020 12:00:42 AM,01/01/2020 12:00:42 AM,01/01/2020 01:37:16 AM,NAT,Necessary Action Taken,Y,2U04,034XX Broadway St,70125.0,2,POINT (-90.10840522 29.95996774),NaN,NaN,-90.10840522,29.95996774
2,2,A2190820,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3682445.0,530709.0,01/17/2020 09:18:41 PM,01/17/2020 09:18:41 PM,01/17/2020 09:18:47 PM,01/17/2020 09:18:54 PM,NAT,Necessary Action Taken,N,8B02,N Peters St & Bienville St,70130.0,8,POINT (-90.065113 29.95323762),NaN,NaN,-90.065113,29.95323762
3,3,A2874820,21,COMPLAINT OTHER,2A,21,COMPLAINT OTHER,1J,3737616.0,590067.0,01/23/2020 10:19:48 AM,01/23/2020 10:22:05 AM,01/23/2020 10:31:11 AM,01/23/2020 10:34:35 AM,GOA,GONE ON ARRIVAL,N,7L08,I-10 E,70129.0,7,POINT (-89.88854843 30.11465463),NaN,NaN,-89.88854843,30.11465463
4,4,A2029120,34S,AGGRAVATED BATTERY BY SHOOTING,2C,34S,AGGRAVATED BATTERY BY SHOOTING,2C,3696210.0,551411.0,01/16/2020 05:09:05 PM,01/16/2020 05:09:43 PM,01/16/2020 05:16:07 PM,01/16/2020 10:49:37 PM,RTF,REPORT TO FOLLOW,N,7A01,Chef Menteur Hwy & Downman Rd,70126.0,7,POINT (-90.02090137 30.00973449),NaN,NaN,-90.02090137,30.00973449


In [24]:
traffic = calls_for_service.loc[calls_for_service["TypeText"] == "TRAFFIC INCIDENT"]

In [25]:
traffic.head()

,Unnamed: 0,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,TimeDispatch,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,Type_,TimeArrival,ycoord,xcoord
132,132,F1732620,18,TRAFFIC INCIDENT,2A,18,TRAFFIC INCIDENT,1J,3699719.0,557531.0,06/14/2020 10:03:32 PM,06/14/2020 11:36:22 PM,06/14/2020 11:51:38 PM,06/15/2020 12:00:41 AM,GOA,GONE ON ARRIVAL,N,7D06,I-10 W & Morrison Rd Exit,70126.0,7,POINT (-90.00959234 30.0264518),NaN,NaN,-90.00959234,30.0264518
566,566,D0513120,18,TRAFFIC INCIDENT,1J,20,AUTO ACCIDENT,1E,3685436.0,544718.0,04/05/2020 05:53:31 AM,04/05/2020 06:59:57 AM,04/05/2020 07:03:50 AM,04/05/2020 07:33:02 AM,NAT,Necessary Action Taken,N,3Y02,I-610 W & Elysian Fields Ave Exit,70122.0,3,POINT (-90.05517653 29.99166695),NaN,NaN,-90.05517653,29.99166695
770,770,D0535620,18,TRAFFIC INCIDENT,1J,18,TRAFFIC INCIDENT,1J,3694280.0,549552.0,04/05/2020 11:39:33 AM,04/05/2020 11:42:38 AM,NaN,04/05/2020 11:50:38 AM,GOA,GONE ON ARRIVAL,N,3U02,Highrise West 3rd,70126.0,3,POINT (-90.02706567 30.0046843),NaN,NaN,-90.02706567,30.0046843
880,880,D0547220,18,TRAFFIC INCIDENT,1A,18,TRAFFIC INCIDENT,1J,3684824.0,538018.0,04/05/2020 02:18:48 PM,04/05/2020 03:23:22 PM,04/05/2020 03:42:49 PM,04/05/2020 03:56:49 PM,GOA,GONE ON ARRIVAL,N,5A04,Elysian Fields Ave & N Claiborne Ave,70117.0,5,POINT (-90.05734351 29.97326333),NaN,NaN,-90.05734351,29.97326333
896,896,D0548820,18,TRAFFIC INCIDENT,1J,18,TRAFFIC INCIDENT,1J,3664008.0,545977.0,04/05/2020 02:43:34 PM,NaN,NaN,04/05/2020 02:45:03 PM,DUP,DUPLICATE,N,3B02,I-610 E & I-10 E Westside,70124.0,3,POINT (-90.12282161 29.99576971),NaN,NaN,-90.12282161,29.99576971


In [26]:
traffic.iloc[0]

Unnamed: 0                                     132
NOPD_Item                                 F1732620
Type                                            18
TypeText                          TRAFFIC INCIDENT
Priority                                        2A
InitialType                                     18
InitialTypeText                   TRAFFIC INCIDENT
InitialPriority                                 1J
MapX                                     3699719.0
MapY                                      557531.0
TimeCreate                  06/14/2020 10:03:32 PM
TimeDispatch                06/14/2020 11:36:22 PM
TimeArrive                  06/14/2020 11:51:38 PM
TimeClosed                  06/15/2020 12:00:41 AM
Disposition                                    GOA
DispositionText                    GONE ON ARRIVAL
SelfInitiated                                    N
Beat                                          7D06
BLOCK_ADDRESS            I-10 W & Morrison Rd Exit
Zip                            

In [27]:
traffic_map = folium.Map(location=[30, -90.15], tiles="Cartodb dark_matter", zoom_start=10.5)

traffic_clean = traffic[["ycoord", "xcoord"]].dropna()

heat_data = [[float(i) for i in nest] for nest in traffic_clean.values.tolist()]
heat_data = [[s[1], s[0]] for s in heat_data]

heat_data

plugins.HeatMap(heat_data, radius = 10).add_to(traffic_map)
traffic_map